In [1]:
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import transforms, models ,datasets
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch.utils.tensorboard import SummaryWriter

In [ ]:
train_path = 'asl_alphabet_train'
test_path = 'asl_alphabet_valid'

train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(p = 0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

In [ ]:
epochs = 20
batch_size = 32
learning_rate = 0.001
feature_extract = False
use_gpu = True
save_for_mobile = False
save_checkpoint = False

In [ ]:
train_data = datasets.ImageFolder(train_path, transform = train_transforms)
test_data = datasets.ImageFolder(test_path, transform = test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

In [ ]:
model = models.mobilenet_v2(pretrained = True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if use_gpu:
    model.to(device)
    
model.classifier = nn.Sequential(
    nn.Dropout(p = 0.2),
    nn.Linear(in_features = 1280, out_features = 29)
) 

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.5)

loss_fn = nn.CrossEntropyLoss()

if use_gpu is True:
    model.to(device)


In [ ]:
writer = SummaryWriter()

In [ ]:
def train():
    model.train()
    
    train_loss = 0
    correct = 0
    
    for index, value in enumerate(train_loader):
        if use_gpu is True:
            inputs, labels = value[0].to(device), value[1].to(device)
        else:
            inputs, labels = value[0], value[1]
        
        optimizer.zero_grad()
        
        output = model(inputs)
        
        loss = loss_fn(output, labels)
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
        _, predicted = torch.max(output.data, 1)
               
        correct += (predicted == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_accuracy = (100. * correct / len(train_loader.dataset))
    
    return [train_loss, train_accuracy]

In [ ]:
def test():
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for index, value in enumerate(test_loader):
            if use_gpu is True:
                inputs, labels = value[0].to(device), value[1].to(device)
            else:
                inputs, labels = value[0], value[1]
            
            output = model(inputs)
            
            test_loss += loss_fn(output, labels).item()
            
            _, predicted = torch.max(output.data, 1)
                
            correct += (predicted == labels).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = (100. * correct / len(test_loader.dataset))
    
    return [test_accuracy, test_loss]

In [ ]:
for epoch in range(1, epochs + 1):
    print('Started epoch: ' + str(epoch))

    loss, accuracy = train(step)
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Accuracy/train', accuracy, epoch)
    
    last_accuracy, test_loss = test(epoch)
    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', last_accuracy, epoch)
    
    torch.save(model, 'model_epoch_' + str(epoch) + '.pt')
    print('Saved a new model on epoch ' + str(epoch))
        
    if save_for_mobile is True and use_gpu is False:
        example = torch.rand(1, 3, 224, 224)
        traced_script_module = torch.jit.trace(model, example)
        torchscript_model_optimized = optimize_for_mobile(traced_script_module)
        torchscript_model_optimized.save('model_mob_epoch_' + str(epoch) + '.pt')
        print('Saved a new mobile optimized model on epoch ' + str(epoch))

    if save_checkpoint is True:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'model_checkpoint' + str(epoch) + '.pt')
        print('Saved a new checkpoint on epoch ' + str(epoch))
    
    scheduler.step()  

print('End')